# Cross Validation for IDW Interpolation 
## Task 2A (continuous & discrete)

This document includes Python codes that conduct cross validation (CV) for Inverse Distance Weighting (IDW) Interpolation on water quality parameters, including 6 water quality parameters in arcpy environment:
- Dissolved oxygen (DO_mgl)
- Salinity (Sal_ppt)
- Turbidity (Turb_ntu)
- Temperature (T_c)
- Secchi (Secc_m)
- Total Nitrogen (TN_mgl) 

The analysis is conducted in the separate water bodies:
- Guana Tolomato Matanzas (GTM)
- Estero Bay (EB)
- Charlotte Harbor (CH)
- Biscayne Bay (BB)
- Big Bend Seagrasses (BBS)

**Tasks:**  

- **Task 2A Calculate the RMSE and Mean Error (ME) for IDW results using both continuous and discrete data**

- Task 2B Calculate the RMSE and Mean Error (ME) for IDW results using continuous data.

Time periods one year before and after storm event for Task 2A tests (seasons).
<br>
<div style="text-align: left;">
    <img src="misc/TimePeriods.png" style="display: block; margin-left: 0; margin-right: auto; width: 600px;"/>
</div>

Summary of IDK and RK Accuracy Assessments.
<br>
<div style="text-align: left;">
    <img src="misc/Table3.png" style="display: block; margin-left: 0; margin-right: auto; width: 600px;"/>
</div>

**Contents:**
* [1. Data Preprocess](#reg_preprocessing)
    * [1.1 Subsetting Dataset](#reg_subset)
    * [1.2 Preview Dataset](#reg_preview)
    * [1.3 Fill Unique ID](#reg_id)
* [2. Create Shapefile](#reg_create_shp)
* [3. Cross Validation for IDW](#reg_cv_idw)

In [3]:
import pandas as pd
import numpy as np
import arcpy
from arcpy.sa import *
import os
import math

import importlib
import sys
# path = r'C:/Users/cong1/WQ/IDW/git/misc'
path = r'E:\Projects\SEACAR_WQ_2024\git\misc'

sys.path.insert(0, path)
import IDW
importlib.reload(IDW)

import pyproj

# define scratch folder to avoid overwritting from parallel threats
arcpy.env.scratchWorkspace = r"E:\Projects\SEACAR_WQ_2024\scratch/IDW_all"

# 1. Data Preprocessing <a class="anchor" id="reg_preprocessing"></a>

### Define water quality files

In [4]:
gis_path = r'E:/Projects/SEACAR_WQ_2024/GIS_Data/'

dfDis = pd.read_csv(gis_path + 'OEAT_Discrete_WQ-2024-Jan-16.csv', low_memory=False)
dfCon = pd.read_csv(gis_path + 'OEAT_Continuous_WQ-2024-Jan-16.csv', low_memory=False)

dfAll = pd.concat([dfDis, dfCon], ignore_index=True)

## 1.1 Subsetting Data <a class="anchor" id="reg_subset"></a>

### Include the time period from 9 am to 17 pm in a day

In [5]:
# Convert string to datetime
dfAll['SampleDate'] = pd.to_datetime(dfAll['SampleDate'], format='%b %d %Y %I:%M%p')

# Include date from 9:00 am to 17:00 pm
start_time = '09:00'
end_time = '17:00'

dfAllTime = dfAll[dfAll['SampleDate'].dt.time.between(pd.to_datetime(start_time).time(), pd.to_datetime(end_time).time())]
dfAllTime.head()

,RowID,ProgramID,ParameterName,ParameterUnits,ProgramLocationID,ActivityType,SampleDate,Year,Month,RelativeDepth,ResultValue,Latitude_DD,Longitude_DD,ManagedAreaName,AreaID,SEACAR_QAQCFlagCode,WaterBody,WbodyAcronym,Season
0,5062527,5014,Salinity,ppt,GTMMKNUT,Field,2017-08-03 11:24:00,2017,8,Surface,0.34,30.160736,-81.360278,Guana Tolomato Matanzas National Estuarine Res...,20,6Q,Guana Tolomato Matanzas,GTM,Summer
1,5062528,5014,Salinity,ppt,GTMMKNUT,Field,2017-09-20 09:06:00,2017,9,Surface,0.32,30.160736,-81.360278,Guana Tolomato Matanzas National Estuarine Res...,20,6Q,Guana Tolomato Matanzas,GTM,Fall
2,5062529,5014,Secchi Depth,m,GTMMKNUT,Field,2017-11-02 13:11:00,2017,11,Surface,1.20,30.160736,-81.360278,Guana Tolomato Matanzas National Estuarine Res...,20,6Q,Guana Tolomato Matanzas,GTM,Fall
3,5062606,5014,Salinity,ppt,GTMMKNUT,Field,2017-10-18 12:52:00,2017,10,Surface,0.34,30.160736,-81.360278,Guana Tolomato Matanzas National Estuarine Res...,20,6Q,Guana Tolomato Matanzas,GTM,Fall
4,5062607,5014,Salinity,ppt,GTMMKNUT,Field,2018-04-24 10:56:00,2018,4,Surface,0.33,30.160736,-81.360278,Guana Tolomato Matanzas National Estuarine Res...,20,6Q,Guana Tolomato Matanzas,GTM,Spring


### Aggregate observations at the same location into an average point

In [6]:
dfAll_Mean = dfAllTime.groupby(['WaterBody','ParameterName','ParameterUnits', 'Year','Season','Latitude_DD','Longitude_DD','WbodyAcronym'])["ResultValue"].agg("mean").reset_index()
dfAll = dfAll_Mean

,WaterBody,ParameterName,ParameterUnits,Year,Season,Latitude_DD,Longitude_DD,WbodyAcronym,ResultValue
0,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.287817,-83.166083,BBS,5.849359
1,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.813933,-83.628917,BBS,6.660736
2,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.101817,-83.076467,BBS,7.408284
3,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.287817,-83.166083,BBS,6.454961
4,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.813933,-83.628917,BBS,7.590892
...,...,...,...,...,...,...,...,...,...
24379,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,29.906941,-81.298931,GTM,28.100000
24380,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.025360,-81.370918,GTM,29.150000
24381,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.026440,-81.369403,GTM,29.500000
24382,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.033611,-81.353027,GTM,29.766667


### Convert coordinate system to EPSG: 3086

In [8]:
# Define the EPSG codes for source (EPSG:4326) and target (EPSG:3086) coordinate systems
source_epsg = 'EPSG:4326'
target_epsg = 'EPSG:3086'

# Create a PyProj Transformer for the conversion
transformer = pyproj.Transformer.from_crs(source_epsg, target_epsg, always_xy=True)

# Define a function to apply the transformation to each row of the DataFrame
def transform_coordinates(row):
    x, y = transformer.transform(row['Longitude_DD'], row['Latitude_DD'])
    return pd.Series({'x': x, 'y': y})

# Apply the transformation function to the DataFrame and create new columns for the converted coordinates
dfAll[['x', 'y']] = dfAll.apply(transform_coordinates, axis=1)

### Save aggregated data to csv file

In [9]:
dfAll.to_csv(gis_path + 'OEAT_All_WQ-2024-Jan-16.csv', index=False)

In [10]:
# Check number of points in a specific run
# dfAll[(dfAll['WaterBody'] == 'Charlotte Harbor') & (dfAll['Year'] == 2018) & (dfAll['ParameterName'] == 'Total Nitrogen') & (dfAll['Season'] == 'Spring')].shape

### Preset abbreviation for waterbody and parameter name

In [11]:
area_shortnames = {
    'Guana Tolomato Matanzas': 'GTM',
    'Estero Bay': 'EB',
    'Charlotte Harbor': 'CH',
    'Biscayne Bay': 'BB',
    'Big Bend Seagrasses':'BBS'
}

param_shortnames = {
    'Salinity': 'Sal_ppt',
    'Total Nitrogen': 'TN_mgl',
    'Dissolved Oxygen': 'DO_mgl',
    'Turbidity':'Turb_ntu',
    'Secchi Depth':'Secc_m',
    'Water Temperature':'T_c'
}

### Define the barrier files

In [12]:
barrier_folder = os.path.join(gis_path, 'Barriers')
barrier_folder

barriers = []
for file in os.listdir(barrier_folder):
    if file.endswith(".shp"):
        barriers.append(os.path.join(barrier_folder, file))

for barrier in barriers:
    print(barrier)

E:/Projects/SEACAR_WQ_2024/GIS_Data/Barriers\BBS_Barriers.shp
E:/Projects/SEACAR_WQ_2024/GIS_Data/Barriers\BB_Barriers.shp
E:/Projects/SEACAR_WQ_2024/GIS_Data/Barriers\CH_Barriers.shp
E:/Projects/SEACAR_WQ_2024/GIS_Data/Barriers\EB_Barriers.shp
E:/Projects/SEACAR_WQ_2024/GIS_Data/Barriers\GTM_Barriers.shp


### Define waterbody boundary for spatial extent and masking

In [13]:
waterbody_extent = os.path.join(gis_path, 'OEAT_Waterbody_Boundaries', 'OEAT_Waterbody_Boundary.shp')

unique_waterbodies = []
with arcpy.da.SearchCursor(waterbody_extent, ['WaterbodyA']) as cursor:
    for row in cursor:
        unique_waterbodies.append(row[0])

print("Unique Waterbodies:", unique_waterbodies)

Unique Waterbodies: ['BBS', 'BB', 'CH', 'EB', 'GTM']


### Define interested study periods and parameters

In [14]:
seasons_all = pd.read_csv(gis_path + 'Seasons_all.csv', low_memory=False)

### Define output folders

In [15]:
shpAll_folder = gis_path + r"shapefiles_All" 
idwAll_folder = gis_path + r"idw_All"

# 1.2 Preview Dataset <a class="anchor" id="reg_preview"></a>

In [16]:
dfAll

,WaterBody,ParameterName,ParameterUnits,Year,Season,Latitude_DD,Longitude_DD,WbodyAcronym,ResultValue,x,y
0,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.287817,-83.166083,BBS,5.849359,480883.595102,587084.408007
1,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.813933,-83.628917,BBS,6.660736,435816.660743,645279.231505
2,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.101817,-83.076467,BBS,7.408284,489730.191291,566492.917816
3,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.287817,-83.166083,BBS,6.454961,480883.595102,587084.408007
4,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.813933,-83.628917,BBS,7.590892,435816.660743,645279.231505
...,...,...,...,...,...,...,...,...,...,...,...
24379,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,29.906941,-81.298931,GTM,28.100000,660458.612741,658403.260621
24380,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.025360,-81.370918,GTM,29.150000,653237.855737,671395.778870
24381,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.026440,-81.369403,GTM,29.500000,653381.222140,671518.795530
24382,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.033611,-81.353027,GTM,29.766667,654941.244725,672348.384187


## 1.3 Fill NaN RowID with unique ID, IDW function needs unique ID <a class="anchor" id="reg_id"></a>

In [17]:
IDW.fill_nan_rowids(dfAll, 'RowID')

# Keep RowID as integer
dfAll['RowID'] = dfAll['RowID'].astype(int)
dfAll

,WaterBody,ParameterName,ParameterUnits,Year,Season,Latitude_DD,Longitude_DD,WbodyAcronym,ResultValue,x,y,RowID
0,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.287817,-83.166083,BBS,5.849359,480883.595102,587084.408007,1
1,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.813933,-83.628917,BBS,6.660736,435816.660743,645279.231505,2
2,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.101817,-83.076467,BBS,7.408284,489730.191291,566492.917816,3
3,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.287817,-83.166083,BBS,6.454961,480883.595102,587084.408007,4
4,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.813933,-83.628917,BBS,7.590892,435816.660743,645279.231505,5
...,...,...,...,...,...,...,...,...,...,...,...,...
24379,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,29.906941,-81.298931,GTM,28.100000,660458.612741,658403.260621,24380
24380,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.025360,-81.370918,GTM,29.150000,653237.855737,671395.778870,24381
24381,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.026440,-81.369403,GTM,29.500000,653381.222140,671518.795530,24382
24382,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.033611,-81.353027,GTM,29.766667,654941.244725,672348.384187,24383


# 2. Create Shapefile <a class="anchor" id="reg_create_shp"></a>

In [18]:
# Empty the shapefile folder
IDW.delete_all_files(shpAll_folder)

In [19]:
# Merge interested with latitude and longitude columns
seasons_all_coord = IDW.merge_with_lat_long(seasons_all, dfAll)
seasons_all_coord

,WaterBody,Year,Season,Parameter,Filename,NumDataPoints,RMSE,ME,x,y,RowID,ResultValue
0,Guana Tolomato Matanzas,2015,Fall,Total Nitrogen,NaN,0,NaN,NaN,NaN,NaN,<NA>,NaN
1,Guana Tolomato Matanzas,2015,Winter,Total Nitrogen,NaN,0,NaN,NaN,NaN,NaN,<NA>,NaN
2,Guana Tolomato Matanzas,2016,Spring,Total Nitrogen,NaN,0,NaN,NaN,NaN,NaN,<NA>,NaN
3,Guana Tolomato Matanzas,2016,Summer,Total Nitrogen,NaN,0,NaN,NaN,NaN,NaN,<NA>,NaN
4,Guana Tolomato Matanzas,2016,Fall,Total Nitrogen,NaN,0,NaN,NaN,NaN,NaN,<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4682,Big Bend Seagrasses,2022,Spring,Water Temperature,E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_out...,27,-1.797693e+308,-1.797693e+308,374832.258094,689623.109934,4393,20.533333
4683,Big Bend Seagrasses,2022,Spring,Water Temperature,E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_out...,27,-1.797693e+308,-1.797693e+308,371015.252225,692080.963200,4394,20.740000
4684,Big Bend Seagrasses,2022,Spring,Water Temperature,E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_out...,27,-1.797693e+308,-1.797693e+308,401894.709931,699334.789795,4395,19.500000
4685,Big Bend Seagrasses,2022,Spring,Water Temperature,E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_out...,27,-1.797693e+308,-1.797693e+308,401457.471573,702258.561096,4396,20.500000


In [20]:
IDW.create_shp_season(seasons_all_coord, shpAll_folder)

Number of data rows for BBS, DO_mgl, 2020, Fall: 26
Shapefile for BBS: DO_mgl for year 2020 and season Fall has been saved as SHP_BBS_DO_mgl_2020_Fall.shp
Number of data rows for BBS, Sal_ppt, 2020, Fall: 21
Shapefile for BBS: Sal_ppt for year 2020 and season Fall has been saved as SHP_BBS_Sal_ppt_2020_Fall.shp
Number of data rows for BBS, Secc_m, 2020, Fall: 27
Shapefile for BBS: Secc_m for year 2020 and season Fall has been saved as SHP_BBS_Secc_m_2020_Fall.shp
Number of data rows for BBS, TN_mgl, 2020, Fall: 23
Shapefile for BBS: TN_mgl for year 2020 and season Fall has been saved as SHP_BBS_TN_mgl_2020_Fall.shp
Number of data rows for BBS, Turb_ntu, 2020, Fall: 26
Shapefile for BBS: Turb_ntu for year 2020 and season Fall has been saved as SHP_BBS_Turb_ntu_2020_Fall.shp
Number of data rows for BBS, T_c, 2020, Fall: 26
Shapefile for BBS: T_c for year 2020 and season Fall has been saved as SHP_BBS_T_c_2020_Fall.shp
Number of data rows for BBS, DO_mgl, 2020, Summer: 26
Shapefile for BB

Shapefile for BB: Turb_ntu for year 2021 and season Fall has been saved as SHP_BB_Turb_ntu_2021_Fall.shp
Number of data rows for BB, T_c, 2021, Fall: 83
Shapefile for BB: T_c for year 2021 and season Fall has been saved as SHP_BB_T_c_2021_Fall.shp
Number of data rows for BB, DO_mgl, 2021, Summer: 83
Shapefile for BB: DO_mgl for year 2021 and season Summer has been saved as SHP_BB_DO_mgl_2021_Summer.shp
Number of data rows for BB, Sal_ppt, 2021, Summer: 61
Shapefile for BB: Sal_ppt for year 2021 and season Summer has been saved as SHP_BB_Sal_ppt_2021_Summer.shp
Number of data rows for BB, Secc_m, 2021, Summer: 1
Shapefile for BB: Secc_m for year 2021 and season Summer has been saved as SHP_BB_Secc_m_2021_Summer.shp
Number of data rows for BB, TN_mgl, 2021, Summer: 76
Shapefile for BB: TN_mgl for year 2021 and season Summer has been saved as SHP_BB_TN_mgl_2021_Summer.shp
Number of data rows for BB, Turb_ntu, 2021, Summer: 61
Shapefile for BB: Turb_ntu for year 2021 and season Summer has 

Shapefile for CH: Sal_ppt for year 2016 and season Winter has been saved as SHP_CH_Sal_ppt_2016_Winter.shp
Number of data rows for CH, Secc_m, 2016, Winter: 8
Shapefile for CH: Secc_m for year 2016 and season Winter has been saved as SHP_CH_Secc_m_2016_Winter.shp
Number of data rows for CH, TN_mgl, 2016, Winter: 8
Shapefile for CH: TN_mgl for year 2016 and season Winter has been saved as SHP_CH_TN_mgl_2016_Winter.shp
Number of data rows for CH, Turb_ntu, 2016, Winter: 11
Shapefile for CH: Turb_ntu for year 2016 and season Winter has been saved as SHP_CH_Turb_ntu_2016_Winter.shp
Number of data rows for CH, T_c, 2016, Winter: 11
Shapefile for CH: T_c for year 2016 and season Winter has been saved as SHP_CH_T_c_2016_Winter.shp
Number of data rows for CH, DO_mgl, 2017, Fall: 3
Shapefile for CH: DO_mgl for year 2017 and season Fall has been saved as SHP_CH_DO_mgl_2017_Fall.shp
Number of data rows for CH, Sal_ppt, 2017, Fall: 3
Shapefile for CH: Sal_ppt for year 2017 and season Fall has been

Shapefile for EB: Turb_ntu for year 2017 and season Spring has been saved as SHP_EB_Turb_ntu_2017_Spring.shp
Number of data rows for EB, T_c, 2017, Spring: 3
Shapefile for EB: T_c for year 2017 and season Spring has been saved as SHP_EB_T_c_2017_Spring.shp
Number of data rows for EB, DO_mgl, 2017, Summer: 3
Shapefile for EB: DO_mgl for year 2017 and season Summer has been saved as SHP_EB_DO_mgl_2017_Summer.shp
Number of data rows for EB, Sal_ppt, 2017, Summer: 3
Shapefile for EB: Sal_ppt for year 2017 and season Summer has been saved as SHP_EB_Sal_ppt_2017_Summer.shp
No valid data found for area: EB, parameter: Secc_m, year: 2017, and season: Summer
No valid data found for area: EB, parameter: TN_mgl, year: 2017, and season: Summer
Number of data rows for EB, Turb_ntu, 2017, Summer: 3
Shapefile for EB: Turb_ntu for year 2017 and season Summer has been saved as SHP_EB_Turb_ntu_2017_Summer.shp
Number of data rows for EB, T_c, 2017, Summer: 3
Shapefile for EB: T_c for year 2017 and season

Shapefile for GTM: T_c for year 2017 and season Spring has been saved as SHP_GTM_T_c_2017_Spring.shp
Number of data rows for GTM, DO_mgl, 2017, Summer: 15
Shapefile for GTM: DO_mgl for year 2017 and season Summer has been saved as SHP_GTM_DO_mgl_2017_Summer.shp
Number of data rows for GTM, Sal_ppt, 2017, Summer: 16
Shapefile for GTM: Sal_ppt for year 2017 and season Summer has been saved as SHP_GTM_Sal_ppt_2017_Summer.shp
Number of data rows for GTM, Secc_m, 2017, Summer: 9
Shapefile for GTM: Secc_m for year 2017 and season Summer has been saved as SHP_GTM_Secc_m_2017_Summer.shp
Number of data rows for GTM, TN_mgl, 2017, Summer: 13
Shapefile for GTM: TN_mgl for year 2017 and season Summer has been saved as SHP_GTM_TN_mgl_2017_Summer.shp
Number of data rows for GTM, Turb_ntu, 2017, Summer: 4
Shapefile for GTM: Turb_ntu for year 2017 and season Summer has been saved as SHP_GTM_Turb_ntu_2017_Summer.shp
Number of data rows for GTM, T_c, 2017, Summer: 17
Shapefile for GTM: T_c for year 2017

# 3. Cross Validation for IDW <a class="anchor" id="reg_cv_idw"></a>

In [21]:
# Empty the shapefile folder
IDW.delete_all_files(idwAll_folder)

In [24]:
# Select a section of table to process
seasons_slct = seasons_all.iloc[122:]

In [ ]:
# If the number of data points is less than 3，skipping calculate IDW
IDW.idw_interpolation(seasons_slct, shpAll_folder, idwAll_folder, waterbody_extent, barrier_folder)

Processing file: SHP_GTM_Turb_ntu_2016_Spring.shp
Calculated RMSE: 3.187827568816108, ME: -0.8804706172849999 for file: SHP_GTM_Turb_ntu_2016_Spring.shp
File SHP_GTM_Turb_ntu_2016_Spring.shp has completed 4 cross-validation iterations.
Processing file: SHP_GTM_Turb_ntu_2016_Summer.shp
Calculated RMSE: 6.272128869247184, ME: -1.8155141907725003 for file: SHP_GTM_Turb_ntu_2016_Summer.shp
File SHP_GTM_Turb_ntu_2016_Summer.shp has completed 4 cross-validation iterations.
Processing file: SHP_GTM_Turb_ntu_2016_Fall.shp
Calculated RMSE: 2.8354251391460434, ME: -0.5828853993825005 for file: SHP_GTM_Turb_ntu_2016_Fall.shp
File SHP_GTM_Turb_ntu_2016_Fall.shp has completed 4 cross-validation iterations.
Processing file: SHP_GTM_Turb_ntu_2016_Winter.shp
Calculated RMSE: 1.9285901655110298, ME: -0.002729204044999989 for file: SHP_GTM_Turb_ntu_2016_Winter.shp
File SHP_GTM_Turb_ntu_2016_Winter.shp has completed 4 cross-validation iterations.
Processing file: SHP_GTM_Turb_ntu_2017_Spring.shp
Calculat

Processing file: SHP_BBS_Turb_ntu_2021_Winter.shp
Calculated RMSE: 2.408141840326292, ME: -0.45865873329782597 for file: SHP_BBS_Turb_ntu_2021_Winter.shp
File SHP_BBS_Turb_ntu_2021_Winter.shp has completed 35 cross-validation iterations.
Processing file: SHP_BBS_Turb_ntu_2022_Spring.shp
Calculated RMSE: 6.568310924812587, ME: -1.1934374680485624 for file: SHP_BBS_Turb_ntu_2022_Spring.shp
File SHP_BBS_Turb_ntu_2022_Spring.shp has completed 28 cross-validation iterations.
Shapefile not found for: SHP_GTM_Secc_m_2015_Fall.shp
Shapefile not found for: SHP_GTM_Secc_m_2015_Winter.shp
Shapefile not found for: SHP_GTM_Secc_m_2016_Spring.shp
Shapefile not found for: SHP_GTM_Secc_m_2016_Summer.shp
Shapefile not found for: SHP_GTM_Secc_m_2016_Fall.shp
Shapefile not found for: SHP_GTM_Secc_m_2016_Winter.shp
Shapefile not found for: SHP_GTM_Secc_m_2017_Spring.shp
Processing file: SHP_GTM_Secc_m_2017_Summer.shp
Calculated RMSE: 0.2450171214641042, ME: -0.08000678222222225 for file: SHP_GTM_Secc_m_20

Processing file: SHP_CH_T_c_2016_Fall.shp
Calculated RMSE: 0.003570446688730135, ME: 7.960000001361323e-07 for file: SHP_CH_T_c_2016_Fall.shp
File SHP_CH_T_c_2016_Fall.shp has completed 3 cross-validation iterations.
Processing file: SHP_CH_T_c_2016_Winter.shp
Calculated RMSE: 1.2331520850534898, ME: -0.33273384927272787 for file: SHP_CH_T_c_2016_Winter.shp
File SHP_CH_T_c_2016_Winter.shp has completed 11 cross-validation iterations.
Processing file: SHP_CH_T_c_2017_Spring.shp
Calculated RMSE: 0.16970815355181781, ME: 7.852499983584948e-07 for file: SHP_CH_T_c_2017_Spring.shp
File SHP_CH_T_c_2017_Spring.shp has completed 3 cross-validation iterations.
Processing file: SHP_CH_T_c_2017_Summer.shp
Calculated RMSE: 0.33710148735002976, ME: -1.3955000000009932e-07 for file: SHP_CH_T_c_2017_Summer.shp
File SHP_CH_T_c_2017_Summer.shp has completed 3 cross-validation iterations.
Processing file: SHP_CH_T_c_2017_Fall.shp
Calculated RMSE: 0.20217572385065918, ME: 5.159499991691519e-07 for file: 